FORECASTING DE CSAT SCORE SEMANAL PARA PREDECIR EL POSIBLE COMPORTAMIENTO PARA 10 SEMANAS EMPLEANDO ES-WINTERS

In [1]:
### LIBRERÍAS

import numpy as np
import pandas as pd

import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from statsmodels.tsa.holtwinters import ExponentialSmoothing

from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
from sklearn import metrics
from math import sqrt

import warnings
warnings.filterwarnings("ignore")
from datetime import datetime
import seaborn as sns

In [5]:
### FUENTE DE DATOS Y PREPARACION DE LOS DATOS

ruta = "C:/Users/joey_/Desktop/DESK 2025/DB (RESPALDOS)/CSAT.xlsx"
df = pd.read_excel(ruta, sheet_name="Sheet1")

df["Start of Week"] = pd.to_datetime(df["Start of Week"])
df = df.rename(columns={"% CSAT (LOOKER)": "CSAT"})
df = df.rename(columns={"Start of Week": "Semana"})
df = df.drop("ID", axis=1)
df["CSAT"] = round(df["CSAT"]*1,2)
df.set_index('Semana', inplace=True)
df.tail(5)

,WEEK,CSAT
Semana,,
2023-12-03,49,0.68
2023-12-10,50,0.74
2023-12-17,51,0.71
2023-12-24,52,0.71
2023-12-31,53,0.68


In [6]:
### MODELANDO LA SERIE DE TIEMPO CON HOLT-WINTERS

# Ajustar el modelo de Holt-Winters
model = ExponentialSmoothing(df['CSAT'], trend='add', seasonal='add', seasonal_periods=13)
model_fit = model.fit()

# Realizar la predicción para las próximas 10 semanas
forecast = model_fit.forecast(steps=10)

# Crear un DataFrame con las predicciones
forecast_dates = pd.date_range(start=df.index[-1] + pd.Timedelta(weeks=1), periods=10, freq='W')
forecast_df = pd.DataFrame({'Semana': forecast_dates, 'CSAT_Predicho': forecast})
forecast_df.set_index('Semana', inplace=True)
forecast_df.head(3)

,CSAT_Predicho
Semana,
2024-01-07,0.682582
2024-01-14,0.662735
2024-01-21,0.679351


In [7]:
### SPLITEANDO LOS DATOS PARA ENTRENAMIENTO Y PRUEBA

train = df['CSAT'][:-10]
test = df['CSAT'][-10:]



In [8]:
### AJUSTE DEL MODELO HOLT-WINTERS A LOS DATOS DE ENTRENAMIENTO, PREDICCIÓN Y CÁLCULO DEL RMSE

model_train = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=13)
model_train_fit = model_train.fit()

# Realizar la predicción en el conjunto de prueba
pred_test = model_train_fit.forecast(steps=10)

# Calcular el RMSE
rmse = sqrt(mean_squared_error(test, pred_test))
print(f'RMSE: {rmse}')

RMSE: 0.04189457169532319


In [9]:
### PREDICCION DE LOS DATOS HISTÓRICOS CON EL MODELO

fitted_values = model_fit.fittedvalues

# Calcular los errores de predicción en el conjunto de entrenamiento
errors = train - model_train_fit.fittedvalues

# Calcular la desviación estándar de los errores
std_errors = np.std(errors)

# Calcular el intervalo de confianza del 95% para las predicciones futuras
forecast_ci_lower = forecast - 1.96 * std_errors
forecast_ci_upper = forecast + 1.96 * std_errors

# Crear un DataFrame para el intervalo de confianza
forecast_ci_df = pd.DataFrame({
    'Semana': forecast_dates,
    'CSAT_Predicho': forecast,
    'CI_Lower': forecast_ci_lower,
    'CI_Upper': forecast_ci_upper
})
forecast_ci_df.set_index('Semana', inplace=True)

In [10]:
### DATAFRAME CON LOS DATOS HISTÓRICOS Y LAS PREDICCIONES


historical_pred_df = pd.DataFrame({
    'CSAT_Historico': df['CSAT'],
    'CSAT_Predicho': fitted_values
})


# Combinar los datos históricos, predicciones futuras y el intervalo de confianza
combined_df = pd.concat([historical_pred_df, forecast_ci_df], axis=0)

combined_df.tail(3)

,CSAT_Historico,CSAT_Predicho,CI_Lower,CI_Upper
Semana,,,,
2024-02-25,NaN,0.710993,0.656160,0.765826
2024-03-03,NaN,0.686085,0.631252,0.740918
2024-03-10,NaN,0.688383,0.633550,0.743216


In [12]:
### VISUALIZACIÓN DEL MODELO HOLT-WINTERS

import plotly.graph_objects as go

# Crear la figura
fig = go.Figure()

# Agregar los valores históricos
fig.add_trace(go.Scatter(
    x=combined_df.index,
    y=combined_df['CSAT_Historico'],
    mode='lines',
    name='CSAT Histórico',
    line=dict(color='blue')
))

# Agregar los valores predichos para los datos históricos
fig.add_trace(go.Scatter(
    x=combined_df.index,
    y=combined_df['CSAT_Predicho'],
    mode='lines',
    name='CSAT Predicho (Actual)',
    line=dict(color='green', dash='dash')
))

# Agregar las predicciones futuras
fig.add_trace(go.Scatter(
    x=forecast_ci_df.index,
    y=forecast_ci_df['CSAT_Predicho'],
    mode='lines',
    name='CSAT Predicho (Futuro)',
    line=dict(color='red')
))

# Agregar el intervalo de confianza del 95%
fig.add_trace(go.Scatter(
    x=forecast_ci_df.index,
    y=forecast_ci_df['CI_Upper'],
    mode='lines',
    name='Límite Superior (95% IC)',
    line=dict(color='gray', width=0.5)
))
fig.add_trace(go.Scatter(
    x=forecast_ci_df.index,
    y=forecast_ci_df['CI_Lower'],
    mode='lines',
    name='Límite Inferior (95% IC)',
    line=dict(color='gray', width=0.5),
    fill='tonexty',
    fillcolor='rgba(200, 100, 200, 0.1)'
))

# Personalizar el gráfico
fig.update_layout(
    title='Predicción del CSAT Score utilizando Holt-Winters con Intervalo de Confianza del 95%',
    xaxis_title='Fecha',
    yaxis_title='CSAT Score',
    hovermode='x unified',
    template="plotly_dark",
    legend=dict(
        orientation="h",
        y=-0.3,
        x=0.5,
        xanchor="center"
    ),
    showlegend=True
)

# Mostrar el gráfico
fig.show()